In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from pathlib import Path
import time
import os
import cv2
from sklearn.model_selection import train_test_split

## Load data

In [2]:
data_dir = "/kaggle/input/handwritten-math-symbols/dataset"
os.listdir(data_dir)

['7',
 '2',
 '5',
 'div',
 '8',
 'x',
 '0',
 'y',
 'z',
 'add',
 '3',
 'eq',
 'dec',
 'sub',
 '1',
 '4',
 '9',
 'mul',
 '6',
 '.directory']

### Setting batch size and the input image size

In [3]:
batch_size = 128
img_row = 28
img_col = 28
channel = 1

### Setting labels for each folder names

In [4]:
labels = {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'add': 10, 'dec': 11, 'div': 12, 'eq': 13, 'mul': 14, 'sub': 15, 'x':16, 'y': 17, 'z': 18, '[': 19, ']': 20}
label = list(labels.keys())[:-2]

num_classes = len(label)
print("Labels dict: ", labels)
print("Labels list: ", label)
print("Num of classes: ", num_classes)

Labels dict:  {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'add': 10, 'dec': 11, 'div': 12, 'eq': 13, 'mul': 14, 'sub': 15, 'x': 16, 'y': 17, 'z': 18, '[': 19, ']': 20}
Labels list:  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'add', 'dec', 'div', 'eq', 'mul', 'sub', 'x', 'y', 'z']
Num of classes:  19


### A function that returns the image to be trained on. (reference to: ➗ Handwritten Equation Solver ➗)
It takes inverse of the image, uses threshold binary, take max contour from the image

In [5]:
# default take inverse, threshold binary, max contour
def get_image(file):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img = ~img
    _, thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    contour = sorted(contours, key = lambda ctr: cv2.boundingRect(ctr)[0])

    a = int(28)
    b = int(28)
    maxi = 0
    
    for c in contour:
        x,y,a,b=cv2.boundingRect(c)
        
        maxi=max(a*b,maxi)
        if maxi==a*b:
            x_max=x
            y_max=y
            w_max=a
            h_max=b

    im_crop = thresh[y_max:y_max+h_max+10, x_max:x_max+w_max+10]
    im_resize = cv2.resize(im_crop,(28,28))
#     cv2.rectangle(img, (x_max, y_max), (x_max + w_max, y_max + h_max), (0, 255, 0), 2)
#     plt.imshow(img)
    im_resize = np.reshape(im_resize,(784))
    return im_resize

Take inverse of the image, threshold binary

In [6]:
# take inverse, threshold binary
def get_image_2(file):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img = ~img
    _, thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)

    im_crop = thresh[:, :]
    im_resize = cv2.resize(im_crop,(28,28))
    im_resize = np.reshape(im_resize,(784))
    return im_resize

Take inverse of the image, threshold zero

In [7]:
# take inverse, threshold tozero
def get_image_3(file):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img = ~img
    _, thresh = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)

    im_crop = thresh[:, :]
    im_resize = cv2.resize(im_crop,(28,28))
    im_resize = np.reshape(im_resize,(784))
    return im_resize

take inverse of the image

In [8]:
# take inverse
def get_image_4(file):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img = ~img
    im_resize = cv2.resize(img,(28,28))
    im_resize = np.reshape(im_resize,(784))
    return im_resize

Modifying the reference code, take inverse of the image, threshold binary, merge all contours instead of taking just max, and make the contour close to square using threshold if not square

In [43]:
# modify the default, take inverse, threshold binary, merge all contours, make it square
inc_thresh = 0.6
def get_image_5(file):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img = ~img
    _, thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    contour = sorted(contours, key = lambda ctr: cv2.boundingRect(ctr)[0])

    a = int(28)
    b = int(28)
    x_max = np.Inf
    y_max = np.Inf
    w_max = 0
    h_max = 0
    
    l,w = 0,0
    
    for c in contour:
        x,y,a,b=cv2.boundingRect(c)
        
        x_max=min(x_max, x)
        y_max=min(y_max, y)
        w_max=max(x_max + w_max, x + a) - x_max
        h_max=max(y_max + h_max, y + b) - y_max

    add_x = 0
    add_y = 0
    if(w_max > h_max and h_max < inc_thresh * w_max):
        add_y = round((inc_thresh * w_max - h_max) * inc_thresh)
    if(h_max > w_max and w_max < inc_thresh * h_max):
        add_x = round((inc_thresh * h_max - w_max) * inc_thresh)
    
    x = max(0, x_max - 5 - add_x)
    y = max(0, y_max - 5 - add_y)
    xa = min(len(img[0]), x_max + w_max + 5 + add_x)
    yb = min(len(img), y_max + h_max + 5 + add_y)

    im_crop = thresh[y:yb, x:xa]
    im_resize = cv2.resize(im_crop,(28,28))
#     cv2.rectangle(img, (x_max, y_max), (x_max + w_max, y_max + h_max), (0, 255, 0), 2)
#     plt.imshow(img)
    im_resize = np.reshape(im_resize,(784))
    return im_resize

Modifying the previous function, use threshold TOZERO

In [40]:
# modify the default, take inverse, threshold binary, merge all contours, make it square
inc_thresh = 0.6
def get_image_6(file):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img = ~img
    _, thresh = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    contour = sorted(contours, key = lambda ctr: cv2.boundingRect(ctr)[0])

    a = int(28)
    b = int(28)
    x_max = np.Inf
    y_max = np.Inf
    w_max = 0
    h_max = 0
    
    l,w = 0,0
    
    for c in contour:
        x,y,a,b=cv2.boundingRect(c)
        
        x_max=min(x_max, x)
        y_max=min(y_max, y)
        w_max=max(x_max + w_max, x + a) - x_max
        h_max=max(y_max + h_max, y + b) - y_max

    add_x = 0
    add_y = 0
    if(w_max > h_max and h_max < inc_thresh * w_max):
        add_y = round((inc_thresh * w_max - h_max) * inc_thresh)
    if(h_max > w_max and w_max < inc_thresh * h_max):
        add_x = round((inc_thresh * h_max - w_max) * inc_thresh)
    
    x = max(0, x_max - 5 - add_x)
    y = max(0, y_max - 5 - add_y)
    xa = min(len(img[0]), x_max + w_max + 5 + add_x)
    yb = min(len(img), y_max + h_max + 5 + add_y)

    im_crop = thresh[y:yb, x:xa]
    im_resize = cv2.resize(im_crop,(28,28))
#     cv2.rectangle(img, (x_max, y_max), (x_max + w_max, y_max + h_max), (0, 255, 0), 2)
#     plt.imshow(img)
    im_resize = np.reshape(im_resize,(784))
    return im_resize

Start picking up image files from each folder and label it

In [54]:
#create data
dat = []
for folder in os.listdir(data_dir):
    if(folder == ".directory"):
        continue
    print("Label: ", folder)
    cat = labels[folder]
    for file in os.listdir(os.path.join(data_dir, folder)):
        if(file == ".directory"):
            continue

        row = get_image(os.path.join(data_dir, folder, file))
        row = np.append(row, cat)
        dat.append(row)

Label:  7
Label:  2
Label:  5
Label:  div
Label:  8
Label:  x
Label:  0
Label:  y
Label:  z
Label:  add
Label:  3
Label:  eq
Label:  dec
Label:  sub
Label:  1
Label:  4
Label:  9
Label:  mul
Label:  6


Create pandas dataframe and save it.

In [55]:
df = pd.DataFrame(dat)
df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,255,255,255,255,196,...,0,0,0,0,0,0,0,0,0,7
1,0,0,0,255,255,255,255,255,255,235,...,0,0,0,0,0,0,0,0,0,7
2,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,7
3,0,0,0,9,9,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,7
4,0,0,20,91,91,103,255,255,255,255,...,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
10067,0,0,0,0,0,0,16,209,255,0,...,0,0,0,0,0,0,0,0,0,6
10068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
10069,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [114]:
suffix_index = 6
suffixes = ["_norm", "_thresh_bin", "_thresh_zero", "_inv", "_merge_bin", "_merge_zero"]
suffix = suffixes[suffix_index - 1]
# suffix = "_thresh_bin"
# suffix = "_thresh_zero"
# suffix = "_inv"
# suffix = "_merge_bin"
# suffix = "_merge_zero"

In [57]:
df.to_csv("/kaggle/working/data" + suffix + ".csv")

Read saved dataframe and prprocess it with normalization and one-hot encoding for labels

In [115]:
df = pd.read_csv("/kaggle/working/data" + suffix + ".csv", index_col=0)

In [116]:
X = df.values[:,:-1]
Y = df.values[:,-1]

X = X.reshape(df.shape[0], img_row, img_col, channel).astype('float32')
X = X / 255
Y = tf.keras.utils.to_categorical(Y, num_classes)

In [61]:
print(X.shape)
print(Y.shape)

(10071, 28, 28, 1)
(10071, 19)


Split the data into train and test

In [117]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15)

In [63]:
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)

(8560, 28, 28, 1)
(8560, 19)
(1511, 28, 28, 1)
(1511, 19)


## Model 1

In [118]:
model = tf.keras.models.Sequential()

# model.add(tf.keras.layers.Rescaling(1./255))
model.add(tf.keras.Input(shape=(img_row, img_col, channel))),

#Adding data augmentation to the model
# model.add(data_augment)
# model.add(tf.keras.layers.RandomRotation(0.1)) # will have range of rotation [-1.6*2pi, 1.6*2pi] i.e. [10, 10] degrees
# model.add(tf.keras.layers.RandomZoom(0.1)) # random zoom of +-10% takes same value for width as well to have the same aspect ratio
# model.add(tf.keras.layers.RandomTranslation(0.1, 0.1))

model.add(tf.keras.layers.Conv2D(32, kernel_size = 3, activation='relu', input_shape = (img_row, img_col, channel), padding = "same"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(64, kernel_size = 3, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
# model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())

# model.add(tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))),
model.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation = "softmax"))

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

In [119]:
epochs = 100
start_time = time.time()
# annealer = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train, Y_train, 
                    epochs=epochs,
                    callbacks = [early_stop],
#                     validation_split = 0.2,
                    validation_data = (X_val, Y_val),
                    verbose = 1)
# history = model.fit(train_ds, 
#                     epochs=epochs,
#                     callbacks = [early_stop],
#                     validation_data=val_ds, # not trained on this data
#                     verbose = 1)

Epoch 1/100


2023-04-29 19:10:33.880453: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_10/dropout_10/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


268/268 [==============================] - 4s 8ms/step - loss: 1.7495 - accuracy: 0.6829 - val_loss: 0.8115 - val_accuracy: 0.8948
Epoch 2/100
268/268 [==============================] - 1s 5ms/step - loss: 0.6880 - accuracy: 0.9014 - val_loss: 0.5590 - val_accuracy: 0.9107
Epoch 3/100
268/268 [==============================] - 1s 5ms/step - loss: 0.4692 - accuracy: 0.9350 - val_loss: 0.3862 - val_accuracy: 0.9451
Epoch 4/100
268/268 [==============================] - 1s 5ms/step - loss: 0.3570 - accuracy: 0.9444 - val_loss: 0.3557 - val_accuracy: 0.9385
Epoch 5/100
268/268 [==============================] - 1s 5ms/step - loss: 0.2943 - accuracy: 0.9492 - val_loss: 0.2766 - val_accuracy: 0.9550
Epoch 6/100
268/268 [==============================] - 1s 5ms/step - loss: 0.2399 - accuracy: 0.9620 - val_loss: 0.2419 - val_accuracy: 0.9530
Epoch 7/100
268/268 [==============================] - 1s 5ms/step - loss: 0.2169 - accuracy: 0.9607 - val_loss: 0.2113 - val_accuracy: 0.9583
Epoch 8/100

In [120]:
model.save("/kaggle/working/model_1" + suffix + ".h5")
model.save_weights("/kaggle/working/model_1_weights" + suffix + ".h5")

## Model 2

Its just trained on all the data

In [121]:
model_2 = tf.keras.models.Sequential()

# model.add(tf.keras.layers.Rescaling(1./255))
model_2.add(tf.keras.Input(shape=(img_row, img_col, channel))),

#Adding data augmentation to the model
# model.add(data_augment)
# model.add(tf.keras.layers.RandomRotation(0.1)) # will have range of rotation [-1.6*2pi, 1.6*2pi] i.e. [10, 10] degrees
# model.add(tf.keras.layers.RandomZoom(0.1)) # random zoom of +-10% takes same value for width as well to have the same aspect ratio
# model.add(tf.keras.layers.RandomTranslation(0.1, 0.1))

model_2.add(tf.keras.layers.Conv2D(32, kernel_size = 3, activation='relu', input_shape = (img_row, img_col, channel), padding = "same"))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_2.add(tf.keras.layers.Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model_2.add(tf.keras.layers.Dropout(0.2))

model_2.add(tf.keras.layers.Conv2D(64, kernel_size = 3, activation='relu'))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(tf.keras.layers.Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
# model.add(tf.keras.layers.Dropout(0.2))

model_2.add(tf.keras.layers.Flatten())

# model.add(tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))),
model_2.add(tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model_2.add(tf.keras.layers.Dense(50, activation='relu'))
model_2.add(tf.keras.layers.Dense(num_classes, activation = "softmax"))

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9)

model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss="categorical_crossentropy", metrics=["accuracy"])

In [122]:
epochs = 100
start_time = time.time()
# annealer = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model_2.fit(X, Y, 
                    epochs=epochs,
                    callbacks = [early_stop],
                    validation_split = 0.2,
                    verbose = 1)

Epoch 1/100


2023-04-29 19:12:21.151537: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_11/dropout_11/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


252/252 [==============================] - 3s 6ms/step - loss: 1.6278 - accuracy: 0.7088 - val_loss: 13.0623 - val_accuracy: 0.1400
Epoch 2/100
252/252 [==============================] - 1s 5ms/step - loss: 0.6423 - accuracy: 0.9080 - val_loss: 15.4355 - val_accuracy: 0.1380
Epoch 3/100
252/252 [==============================] - 1s 5ms/step - loss: 0.4668 - accuracy: 0.9271 - val_loss: 14.5036 - val_accuracy: 0.1385
Epoch 4/100
252/252 [==============================] - 1s 6ms/step - loss: 0.3448 - accuracy: 0.9440 - val_loss: 17.4031 - val_accuracy: 0.1375
Epoch 5/100
252/252 [==============================] - 1s 6ms/step - loss: 0.2759 - accuracy: 0.9542 - val_loss: 16.8756 - val_accuracy: 0.1494
Epoch 6/100
252/252 [==============================] - 1s 5ms/step - loss: 0.2327 - accuracy: 0.9614 - val_loss: 17.2157 - val_accuracy: 0.1325
Epoch 7/100
252/252 [==============================] - 2s 7ms/step - loss: 0.2067 - accuracy: 0.9633 - val_loss: 15.7687 - val_accuracy: 0.1459
Epoc

In [123]:
model_2.save("/kaggle/working/model_2" + suffix + ".h5")
model_2.save_weights("/kaggle/working/model_2_weights" + suffix + ".h5")

In [ ]:
model.evaluate(X_val,Y_val)

In [ ]:
model_2.evaluate(X_val, Y_val)

In [ ]:
def plotplot(history):
    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plot = plt.figure(figsize=(15,10))
    plt.subplot(2, 2, 1)
    plt.plot(accuracy, label = "Training accuracy")
    plt.plot(val_accuracy, label="Validation accuracy")
    plt.legend()
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs Epochs")

    plt.subplot(2,2,2)
    plt.plot(loss, label = "Training loss")
    plt.plot(val_loss, label="Validation loss")
    plt.legend()
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss vs epochs")
    return plot

In [ ]:
plot = plotplot(history)
plot.show()

Read model file from the saved model

In [ ]:
model_file = "/kaggle/working/model_1" + suffix + ".h5"
model = tf.keras.models.load_model(model_file)

## Test on unseen data for individual characters

In [ ]:
test_dir = "/kaggle/input/test-basic-math/test/all/"
list_img = os.listdir(test_dir)
list_img

In [ ]:
def prediction_2(img):
    x = get_image_2(img).reshape(img_row, img_col, channel).astype("float32")
    plt.imshow(x)
    plt.plot()
    x = np.array([x]) / 255
    # print(x.shape)
    
    pred = model.predict(x, verbose=0)[0]
    # print(pred)
    # print(pred.argmax())
    print("Predicted label : ", label[pred.argmax()])
    # print(label[pred.argmax()])

In [ ]:
for img in list_img:
    print("img : ", img)
    prediction_2(test_dir + img)

## Test on equations

In [ ]:
eq_dir = "/kaggle/input/test-basic-math/test/eq/"
eq_list = os.listdir(eq_dir)
eq_list

Used to merge overlapping contours

In [ ]:
def contour_union_2(x, y, xa, yb, key, l):
    x_i = x[key]
    y_i = y[key]
    xa_i = xa[key]
    yb_i = yb[key]
    for j in l:
        x_i = min(x_i, x[j])
        y_i = min(y_i, y[j])
        xa_i = max(xa_i, xa[j])
        yb_i = max(yb_i, yb[j])
    return [x_i, y_i, xa_i, yb_i]

Used to find overlapping contours and merge them into one and return them as individual contour. Use only the x-axis overlap as the equation goes from left to right.

In [ ]:
def merge_all_contours(img, contour):
    contours = []
    contours_x = []
    contours_xa = []
    contours_y = []
    contours_yb = []

    for c in contour:
        x,y,a,b=cv2.boundingRect(c)
        contours.append([x, y, a, b])
        contours_x.append(x)
        contours_xa.append(x + a)
        contours_y.append(y)
        contours_yb.append(y + b)
        # cv2.rectangle(img, (x - 5, y - 5), (x + a + 5, y + b + 5), (0, 255, 0), 2)
    
    overlaps={}

    for i in range(len(contours)):
        for j in range(i + 1, len(contours)):
            if((contours_x[i] <= contours_x[j] and contours_xa[i] >= contours_x[j] and contours_xa[i] <= contours_xa[j]) or # i on left of j
               (contours_x[i] >= contours_x[j] and contours_x[i] <= contours_xa[j] and contours_xa[i] >= contours_xa[j]) or # j on left of i
               (contours_x[i] >= contours_x[j] and contours_xa[i] <= contours_xa[j]) or # i inside of j
               (contours_x[i] <= contours_x[j] and contours_xa[i] >= contours_xa[j])): # j inside of i
                if(i not in overlaps.keys()):
                    overlaps[i] = [j]
                else:
                    overlaps[i].append(j)

    # print(overlaps)

    for key in reversed(overlaps.keys()):
        for ival in overlaps[key]:
            keep = []
            if(ival not in overlaps.keys()):
                continue
            for jval in overlaps[ival]:
                if(jval not in overlaps[key]):
                    overlaps[key].append(jval)
            overlaps[ival] = []
    # print(overlaps)
    
    keys = list(overlaps.keys())
    used_contours = []
    max_contour = []
    for key in keys:
        if(len(overlaps[key]) == 0):
            overlaps.pop(key)
            continue
        overlaps[key].sort()
        used_contours.append(key)
        used_contours.extend(overlaps[key])
        max_contour.append(contour_union_2(contours_x, contours_y, contours_xa, contours_yb, key, overlaps[key]))

    # print(overlaps)
    # print(max_contour)
    # print(used_contours)
        
    new_contour = []
    for i in range(len(contours)):
        if(i in used_contours):
            continue
        x_i = contours_x[i]
        y_i = contours_y[i]
        xa_i = contours_xa[i]
        yb_i = contours_yb[i]
        new_contour.append([x_i, y_i, xa_i, yb_i])

    # print(new_contour)

    new_contour.extend(max_contour)
    new_contour.sort(key = lambda x: x[0])
    
    return new_contour

Delete contours that are too small in size using individual dimension threshold and make the contour more like a square so that the resize does not ruin the image using a threshold that atleast width or height must be atleast 60% of the height or width

In [ ]:
def adjust_contours(new_contour, dims_thresh, inc_thresh):
    new_new_contour = []
    for c in new_contour:
        x,y,xa,yb=c
        l, w = (xa-x), (yb-y)
        contour_dims = (xa-x) * (yb-y)
        # print("countour dimension: ", contour_dims)
        if(l < dims_thresh and w < dims_thresh):
            continue
        add_x = 0
        add_y = 0
        if(l > w and w < inc_thresh * l):
            add_y = round((inc_thresh * l - w) * inc_thresh)
        if(w > l and l < inc_thresh * w):
            add_x = round((inc_thresh * w - l) * inc_thresh)
        # print(add_x, add_y)
        if(contour_dims > dims_thresh):
            # adding contour
            # print("adding contour")
            
            x = max(0, x - 5 - add_x)
            y = max(0, y - 5 - add_y)
            xa = min(len(img[0]), xa + 5 + add_x)
            yb = min(len(img), yb + 5 + add_y)
            new_new_contour.append([x,y,xa,yb])
    return new_new_contour

Get the individual images from the whole equation photo

In [ ]:
def get_image_all_2(file):
    img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    img = ~img
#     _, thresh = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
    _, thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    contour = sorted(contours, key = lambda ctr: cv2.boundingRect(ctr)[0])
    
    new_contour = merge_all_contours(img, contour)
    
    # print("final contours :", new_contour)

    
    dims_thresh = 20
    # remove small contours
    # make contours slightly square
    # increasing the width or length threshold
    inc_thresh = 0.6
    
    final_contour = adjust_contours(new_contour, dims_thresh, inc_thresh)

    # print("new new contour after filtering threshold:", new_new_contour)

    images = []

    for c in final_contour:
        x,y,xa,yb=c
        cv2.rectangle(img, (x, y), (xa, yb), (0, 255, 0), 2)
        images.append(thresh[y : yb, x : xa])
    images = np.array(images)
    # plt.imshow(img)

    return images

Preprocess image by normalizing it

In [ ]:
def process_image(img):
    # print(img.shape)
    temp = cv2.resize(img, (img_row, img_col))
    temp = temp / 255
    temp = np.reshape(temp, (img_row, img_col, 1))
    # return temp
    return np.array([temp])

### Predictions:

In [ ]:
expected = ["5+4", "3+2", "54+3", "42+1", "21-3", "2*4", "2x+5=7", "y=3x+4", "z=3x+4y", "1+2", "4+5", "[4+3]*5", "8/2", "512/128", "52.9/68"]
# expected = ["5add4", "3add2", "54add3", "42add1", "21sub3", "2mul4", "2xadd5eq7", "yeq3xadd4", "zeq3xadd4y", "1add2", "4add5", "[4add3]mul5", "8div2", "512div128", "52dec9div68"]

count=0
for ind in range(len(eq_list)):
    # og_img = cv2.imread(eq_dir + eq_list[ind], cv2.IMREAD_GRAYSCALE)
    # plt.imshow(og_img)
    eq_img = get_image_all_2(eq_dir + eq_list[ind])
    eq_str = ""
    for img in eq_img:
        im = process_image(img)
        pred = model.predict(im, verbose=0)[0]
        lab = label[pred.argmax()]

        # print("Predicted label : ", lab)
        
        if(lab == "eq"):
            lab = "="
        elif(lab == "dec"):
            lab = "."
        elif(lab == "add"):
            lab = "+"
        elif(lab == "sub"):
            lab = "-"
        elif(lab == "div"):
            lab = "/"
        elif(lab == "mul"):
            lab = "*"

        eq_str += lab
    if(expected[ind] == eq_str):
        count += 1
    print(expected[ind], end=" ")
    print(eq_str)
print("Correct predictions: ", count)
# out of 15 images, for thresh binary
# old/model_1_norm: 0
# old/model_1_inv: 5
# old/model_1_thresh: 6
# old/model_2_norm: 0 (not correct)
# old/model_2_inv: 1
# old/model_2_thresh: 2
# new/model_1_inv: 6
# new/model_2_inv: 2
# new/model_3_inv: 3

# out of 15 images, for thresh tozero
# old/model_1_norm: 0
# old/model_1_inv: 5
# old/model_1_thresh: 6
# old/model_2_norm: 0 (not correct)
# old/model_2_inv: 1
# old/model_2_thresh: 2
# new/model_1_inv: 5
# new/model_2_inv: 3
# new/model_3_inv: 3